# 沒有 elements

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import re
import glob

sentences_drop = {
    'YS': ['PROPERTY: YS (MPa)'],
    'YM': ['PROPERTY: Calculated Young modulus (GPa)', 'PROPERTY: Exp. Young modulus (GPa)'],
    'HV': ['PROPERTY: HV'],
    'El': ['PROPERTY: Elongation (%)', 'PROPERTY: Elongation plastic (%)'],
    'UTS': ['PROPERTY: UTS (MPa)'],
    'ExpDensity': ['PROPERTY: Exp. Density (g/cm$^3$)', 'PROPERTY: Calculated Density (g/cm$^3$)'],
    'calcDensity': ['PROPERTY: Calculated Density (g/cm$^3$)', 'PROPERTY: Exp. Density (g/cm$^3$)'],
}


sentences = {
    # "IDENTIFIER: Reference ID": 'Reference ID is {}.',
    "FORMULA": 'Alloy\'s formula is {}.',
    "PROPERTY: Microstructure": 'Microstructure of alloy is {}.',  # 最後 "PROPERTY: BCC/FCC/other" 的句子會合併進來了
    "PROPERTY: Processing method": 'Processing method of the alloy is {}.',
    "PROPERTY: BCC/FCC/other": 'The alloy is {}.',
    "PROPERTY: Grain size ($\mu$m)": 'Alloy\'s grain size is {} microns.',
    "PROPERTY: Exp. Density (g/cm$^3$)": 'Experimental density of the alloy is {} g/cm^3.',
    "PROPERTY: Calculated Density (g/cm$^3$)": 'Calculated density of the alloy is {} g/cm^3.',
    "PROPERTY: HV": 'Hardness of the alloy is {} in Vickers scale.',
    "PROPERTY: Type of test": 'The alloy was tested in {} test.',
    "PROPERTY: Test temperature ($^\circ$C)": 'Alloy as tested at {} Celsius.',
    "PROPERTY: YS (MPa)": 'Yield Strength (YS) of alloy is {}MPa.',                 
    "PROPERTY: UTS (MPa)": 'Ultimate tensile strength (UTS) of alloy is {}MPa.',
    "PROPERTY: Elongation (%)": 'The alloy has {}% elongation.',
    "PROPERTY: Elongation plastic (%)": 'Plastic elongation of the alloy is {}%.',
    "PROPERTY: Exp. Young modulus (GPa)": 'Experimental Young modulus of the alloy is {}GPa.', 
    "PROPERTY: Calculated Young modulus (GPa)": 'Calculated Young modulus of the alloy is {}GPa.',  
    # "PROPERTY: O content (wppm)": 'Oxygen content of alloy is {} wppm.',
    # "PROPERTY: N content (wppm)": 'Nitrogen content of alloy is {} wppm.',
    # "PROPERTY: C content (wppm)": 'Carbon content of alloy is {} wppm.',
    # "REFERENCE: doi": 'The DOI of the paper is {}.',
    # "REFERENCE: year": 'The year of the paper is {}.',
    # "REFERENCE: title": 'The title of the paper is {}.',

    # add
    # "PROPERTY: Elongation fracture (%)": 'Fracture elongation of the alloy is {}%.',
}


processing_methods = {
    'CAST': "Liquid material poured into a mold, solidifying to form the casting. Used for complex shapes, especially for metals or materials like epoxy, concrete, plaster, and clay.",
    'WROUGHT': "Shaping materials through plastic deformation, including rolling, forging, extrusion, or drawing. Commonly applied to metallic alloys for specific shapes, sizes, and mechanical properties.",
    'POWDER': "Production and shaping of metallic alloys as powders. Utilizes techniques like powder production, consolidation, and sintering in powder metallurgy. Offers versatility for controlled composition and complex geometries.",
    'ANNEAL': "Heat treatment process for alloys. Involves heating to a specific temperature, holding, and controlled cooling. Used to relieve stresses, improve machinability, enhance mechanical properties, and alter microstructure."
}


microstructure = {
    "FCC": "Atoms arranged in a cube with one at each corner and one at the center of each face. Packing density ~74%.",
    "BCC": "Atoms arranged in a cube with one at each corner and one at the center. Packing density than FCC ~68%."
}


def detect_target_key(filename):
    """
    自動從檔名抓 target_key
    依據 sentences_drop 的 key 去檔名搜尋
    """
    for key in sentences_drop.keys():
        if key.lower() in filename.lower():
            return key
    return None


def generate_text_from_folder(input_dir, output_dir=None):
    csv_files = glob.glob(f"{input_dir}/*.csv")
    print(f"在資料夾 {input_dir} 中找到 {len(csv_files)} 個 CSV 檔\n")

    for path in csv_files:
        filename = os.path.basename(path)

        # ----------------------------------------------------
        # 自動偵測 target_key
        # ----------------------------------------------------
        target_key = detect_target_key(filename)
        if target_key is None:
            print(f"[警告] 檔名無法判斷 target_key，跳過: {filename}")
            continue

        print(f"讀取: {filename}   → 偵測到 target_key = {target_key}")

        # output_name = filename.replace("_convert.csv", f"_sentence.pkl")
        output_name = filename.replace("_convert.csv", f"_sentence.csv")

        df = pd.read_csv(path)

        # 自動補齊 'PROPERTY: BCC/FCC/other' 欄位
        if "PROPERTY: BCC/FCC/other" not in df.columns and "PROPERTY: Microstructure" in df.columns:
            df["PROPERTY: BCC/FCC/other"] = df["PROPERTY: Microstructure"]
            
        # 移除 target 相關欄位
        cols_to_drop = sentences_drop[target_key]
        df_features = df[[col for col in df.columns if col not in cols_to_drop]]

        df_dict = df_features.to_dict('records')

        all_sent_dicts = []
        for record in df_dict:
            sent_dict = {}
            for prop in record:
                if prop in sentences and pd.notnull(record[prop]):

                    # FORMULA: 只保留主句，不拆元素
                    if prop == "FORMULA":
                        sent_dict[prop] = sentences[prop].format(record[prop])
                        continue

                    # 主句
                    sent_dict[prop] = sentences[prop].format(record[prop])

                    # Processing method sentence
                    if prop == "PROPERTY: Processing method":
                        method = record[prop]
                        sent_dict[prop] = sent_dict[prop] + processing_methods[method]
                        # if method in processing_methods:
                        #     sent_dict[prop + "__extra"] = processing_methods[method]

                    # Microstructure extra info
                    if prop == "PROPERTY: Microstructure":
                        ms = record[prop]
                        sent_dict[prop] = sent_dict[prop] + microstructure[ms]
                        # if ms in microstructure:
                        #     sent_dict[prop + "__extra"] = microstructure[ms]
                        
                    if prop == "PROPERTY: BCC/FCC/other": # 將 "PROPERTY: BCC/FCC/other" 合併進 Microstructure
                        if "PROPERTY: Microstructure" in sent_dict:
                            sent_dict["PROPERTY: Microstructure"] = sent_dict["PROPERTY: Microstructure"] + sent_dict[prop]
            sent_dict.pop("PROPERTY: BCC/FCC/other", None) # 移除已合併的欄位
            all_sent_dicts.append(sent_dict)
        

        # -------------------------
        # 用「原本欄位名」當句子欄位名稱
        # -------------------------
        # 只保留 target_nor 當基底，其他句子欄位後面再加
        df_out = df[['target_nor']].copy()

        # 所有句子欄位名稱（原本的欄位名 + 加上的 __extra / __info 等）
        all_keys = set().union(*all_sent_dicts)

        # 每個 key 就是一個欄位，用原本的欄位名
        for key in all_keys:
            df_out[key] = [d.get(key, np.nan) for d in all_sent_dicts]

        # 把 target_nor 改名成 target，放在第一欄
        df_out = df_out.rename(columns={"target_nor": "target"})
        cols = ['target'] + [c for c in df_out.columns if c != 'target']
        df_out = df_out[cols]

        # 🔥 空值補 Unknown
        df_out = df_out.fillna("Unknown")

        save_path = f"{output_dir}/{output_name}"

        # df_out.to_pickle(save_path)
        df_out.to_csv(save_path, index=False)

        print(f"→ 已輸出: {save_path}")
        print("-----------------------------------------------------------")


# # =======================
# # 使用方式
# # =======================
# if __name__ == "__main__":
#     input_dir = "你指定的資料夾路徑"
#     generate_text_from_folder(input_dir)


In [26]:
# =======================
# 使用方式
# =======================
input_dir = "data"
output_dir = "sentence"
generate_text_from_folder(input_dir, output_dir)

在資料夾 data 中找到 4 個 CSV 檔

讀取: vl_all_data_calcDensity_convert.csv   → 偵測到 target_key = calcDensity
→ 已輸出: sentence/vl_all_data_calcDensity_sentence.csv
-----------------------------------------------------------
讀取: vl_all_data_ExpDensity_convert.csv   → 偵測到 target_key = ExpDensity
→ 已輸出: sentence/vl_all_data_ExpDensity_sentence.csv
-----------------------------------------------------------
讀取: vl_all_data_HV_convert.csv   → 偵測到 target_key = HV
→ 已輸出: sentence/vl_all_data_HV_sentence.csv
-----------------------------------------------------------
讀取: vl_all_data_YS_convert.csv   → 偵測到 target_key = YS
→ 已輸出: sentence/vl_all_data_YS_sentence.csv
-----------------------------------------------------------


# 有 elements

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import re
import glob

sentences_drop = {
    'YS': ['PROPERTY: YS (MPa)'],
    'YM': ['PROPERTY: Calculated Young modulus (GPa)', 'PROPERTY: Exp. Young modulus (GPa)'],
    'HV': ['PROPERTY: HV'],
    'El': ['PROPERTY: Elongation (%)', 'PROPERTY: Elongation plastic (%)'],
    'UTS': ['PROPERTY: UTS (MPa)'],
    'ExpDensity': ['PROPERTY: Exp. Density (g/cm$^3$)'],
}


sentences = {
    "IDENTIFIER: Reference ID": 'Reference ID is {}.',
    "FORMULA": 'Alloy\'s formula is {}.',
    "PROPERTY: Microstructure": 'Microstructure of alloy is {}.',
    "PROPERTY: Processing method": 'Processing method of the alloy is {}.',
    "PROPERTY: BCC/FCC/other": 'The alloy is {}.',
    "PROPERTY: Grain size ($\mu$m)": 'Alloy\'s grain size is {} microns.',
    "PROPERTY: Exp. Density (g/cm$^3$)": 'Experimental density of the alloy is {} g/cm^3.',
    "PROPERTY: Calculated Density (g/cm$^3$)": 'Calculated density of the alloy is {} g/cm^3.',
    "PROPERTY: HV": 'Hardness of the alloy is {} in Vickers scale.',
    "PROPERTY: Type of test": 'The alloy was tested in {} test.',
    "PROPERTY: Test temperature ($^\circ$C)": 'Alloy as tested at {} Celsius.',
    "PROPERTY: YS (MPa)": 'Yield Strength (YS) of alloy is {}MPa.',                 
    "PROPERTY: UTS (MPa)": 'Ultimate tensile strength (UTS) of alloy is {}MPa.',
    "PROPERTY: Elongation (%)": 'The alloy has {}% elongation.',
    "PROPERTY: Elongation plastic (%)": 'Plastic elongation of the alloy is {}%.',
    "PROPERTY: Exp. Young modulus (GPa)": 'Experimental Young modulus of the alloy is {}GPa.', 
    "PROPERTY: Calculated Young modulus (GPa)": 'Calculated Young modulus of the alloy is {}GPa.',  
    "PROPERTY: O content (wppm)": 'Oxygen content of alloy is {} wppm.',
    "PROPERTY: N content (wppm)": 'Nitrogen content of alloy is {} wppm.',
    "PROPERTY: C content (wppm)": 'Carbon content of alloy is {} wppm.',
    "REFERENCE: doi": 'The DOI of the paper is {}.',
    "REFERENCE: year": 'The year of the paper is {}.',
    "REFERENCE: title": 'The title of the paper is {}.',

    # add
    # "PROPERTY: Elongation fracture (%)": 'Fracture elongation of the alloy is {}%.',
}


processing_methods = {
    'CAST': "Liquid material poured into a mold, solidifying to form the casting. Used for complex shapes, especially for metals or materials like epoxy, concrete, plaster, and clay.",
    'WROUGHT': "Shaping materials through plastic deformation, including rolling, forging, extrusion, or drawing. Commonly applied to metallic alloys for specific shapes, sizes, and mechanical properties.",
    'POWDER': "Production and shaping of metallic alloys as powders. Utilizes techniques like powder production, consolidation, and sintering in powder metallurgy. Offers versatility for controlled composition and complex geometries.",
    'ANNEAL': "Heat treatment process for alloys. Involves heating to a specific temperature, holding, and controlled cooling. Used to relieve stresses, improve machinability, enhance mechanical properties, and alter microstructure."
}


microstructure = {
    "FCC": "Atoms arranged in a cube with one at each corner and one at the center of each face. Packing density ~74%.",
    "BCC": "Atoms arranged in a cube with one at each corner and one at the center. Packing density than FCC ~68%."
}

elements = {
    "Al": "Aluminum, atomic number 13, atomic mass 26.98 u. Density 2.70 g/cm³, melts at 660.3°C. FCC microstructure, Young's Modulus ≈ 70 GPa.",
    "Co": "Cobalt, atomic number 27, atomic mass 58.93 u. Density 8.90 g/cm³, melts at 1495°C. HCP microstructure, Young's Modulus ≈ 211 GPa.",
    "Fe": "Iron, atomic number 26, atomic mass 55.85 u. Density 7.87 g/cm³, melts at 1538°C. BCC microstructure, Young's Modulus ≈ 211 GPa.",
    "Ni": "Nickel, atomic number 28, atomic mass 58.69 u. Density 8.91 g/cm³, melts at 1455°C. FCC microstructure, Young's Modulus ≈ 200 GPa.",
    "Si": "Silicon, atomic number 14, atomic mass 28.09 u. Density 2.33 g/cm³, melts at 1414°C. Diamond cubic structure, Young's Modulus ≈ 130 GPa.",
    "Cr": "Chromium, atomic number 24, atomic mass 51.996 u. Density 7.19 g/cm³, melts at 1907°C. BCC microstructure, Young's Modulus ≈ 279 GPa.",
    "Mn": "Manganese, atomic number 25, atomic mass 54.94 u. Density 7.21 g/cm³, melts at 1246°C. BCC microstructure, Young's Modulus ≈ 198 GPa.",
    "Ti": "Titanium, atomic number 22, atomic mass 47.87 u. Density 4.54 g/cm³, melts at 1668°C. HCP structure, Young's Modulus ≈ 116 GPa.",
    "Cu": "Copper, atomic number 29, atomic mass 63.55 u. Density 8.96 g/cm³, melts at 1085°C. FCC structure, Young's Modulus ≈ 130 GPa.",
    "Mo": "Molybdenum, atomic number 42, atomic mass 95.95 u. Density 10.22 g/cm³, melts at 2623°C. BCC structure, Young's Modulus ≈ 329 GPa.",
    "Nb": "Niobium, atomic number 41, atomic mass 92.91 u. Density 8.57 g/cm³, melts at 2468°C. BCC structure, Young's Modulus ≈ 105 GPa.",
    "V": "Vanadium, atomic number 23, atomic mass 50.94 u. Density 6.11 g/cm³, melts at 1910°C. BCC structure, Young's Modulus ≈ 128 GPa.",
    "Zr": "Zirconium, atomic number 40, atomic mass 91.22 u. Density 6.52 g/cm³, melts at 1852°C. HCP structure, Young's Modulus ≈ 88 GPa.",
    "Sn": "Tin, atomic number 50, atomic mass 118.71 u. Density 7.29 g/cm³, melts at 231.9°C. Tetragonal structure, Young's Modulus ≈ 50 GPa.",
    "Ta": "Tantalum, atomic number 73, atomic mass 180.95 u. Density 16.65 g/cm³, melts at 3290°C. BCC structure, Young's Modulus ≈ 200 GPa.",
    "Hf": "Hafnium, atomic number 72, atomic mass 178.49 u. Density 13.31 g/cm³, melts at 2233°C. HCP structure, Young's Modulus ≈ 78 GPa.",
    "W": "Tungsten, atomic number 74, atomic mass 183.84 u. Density 19.25 g/cm³, melts at 3422°C. BCC structure, Young's Modulus ≈ 411 GPa.",
    "Zn": "Zinc, atomic number 30, atomic mass 65.38 u. Density 7.14 g/cm³, melts at 419.5°C. HCP structure, Young's Modulus ≈ 96 GPa.",
    "Re": "Rhenium, atomic number 75, atomic mass 186.21 u. Density 21.04 g/cm³, melts at 3186°C. HCP structure, Young's Modulus ≈ 463 GPa.",
    "Mg": "Magnesium, atomic number 12, atomic mass 24.31 u. Density 1.74 g/cm³, melts at 650°C. HCP structure, Young's Modulus ≈ 45 GPa.",
    "Pd": "Palladium, atomic number 46, atomic mass 106.42 u. Density 12.02 g/cm³, melts at 1554°C. FCC structure, Young's Modulus ≈ 121 GPa."
}

def detect_target_key(filename):
    """
    自動從檔名抓 target_key
    依據 sentences_drop 的 key 去檔名搜尋
    """
    for key in sentences_drop.keys():
        if key.lower() in filename.lower():
            return key
    return None


def generate_text_from_folder(input_dir, output_dir=None):
    csv_files = glob.glob(f"{input_dir}/*.csv")
    print(f"在資料夾 {input_dir} 中找到 {len(csv_files)} 個 CSV 檔\n")

    for path in csv_files:
        filename = os.path.basename(path)

        # ----------------------------------------------------
        # 自動偵測 target_key
        # ----------------------------------------------------
        target_key = detect_target_key(filename)
        if target_key is None:
            print(f"[警告] 檔名無法判斷 target_key，跳過: {filename}")
            continue

        print(f"讀取: {filename}   → 偵測到 target_key = {target_key}")

        # output_name = filename.replace("_convert.csv", f"_sentence.pkl")
        output_name = filename.replace("_convert.csv", f"_sentence.csv")

        df = pd.read_csv(path)

        # 移除 target 相關欄位
        cols_to_drop = sentences_drop[target_key]
        df_features = df[[col for col in df.columns if col not in cols_to_drop]]

        df_dict = df_features.to_dict('records')

        all_sent_dicts = []
        for record in df_dict:
            sent_dict = {}
            for prop in record:
                if prop in sentences and pd.notnull(record[prop]):

                    # 主句
                    sent_dict[prop] = sentences[prop].format(record[prop])

                    # FORMULA 額外句子（若你要的話）
                    if prop == "FORMULA":
                        s = record[prop]
                        floats = re.findall(r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?', s)
                        floats = [float(f[0]) for f in floats]
                        strings = re.split(r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?', s)
                        strings = [st.strip() for st in strings if st and not st.replace('.', '', 1).isdigit()]

                        for i in range(len(strings)):
                            if strings[i] in elements:
                                key_base = f"{prop}__{strings[i]}"
                                sent_dict[key_base] = f"Alloy has {floats[i]} atoms of {strings[i]}."
                                sent_dict[key_base + "__info"] = elements[strings[i]]

                    # Processing method sentence
                    if prop == "PROPERTY: Processing method":
                        method = record[prop]
                        if method in processing_methods:
                            sent_dict[prop + "__extra"] = processing_methods[method]

                    # Microstructure
                    if prop == "PROPERTY: Microstructure":
                        ms = record[prop]
                        if ms in microstructure:
                            sent_dict[prop + "__extra"] = microstructure[ms]

            all_sent_dicts.append(sent_dict)

        # -------------------------
        # 用「原本欄位名」當句子欄位名稱
        # -------------------------
        # 只保留 target_nor 當基底，其他句子欄位後面再加
        df_out = df[['target_nor']].copy()

        # 所有句子欄位名稱（原本的欄位名 + 加上的 __extra / __info 等）
        all_keys = set().union(*all_sent_dicts)

        # 每個 key 就是一個欄位，用原本的欄位名
        for key in all_keys:
            df_out[key] = [d.get(key, np.nan) for d in all_sent_dicts]

        # 把 target_nor 改名成 target，放在第一欄
        df_out = df_out.rename(columns={"target_nor": "target"})
        cols = ['target'] + [c for c in df_out.columns if c != 'target']
        df_out = df_out[cols]

        save_path = f"{output_dir}/{output_name}"

        # df_out.to_pickle(save_path)
        df_out.to_csv(save_path, index=False)

        print(f"→ 已輸出: {save_path}")
        print("-----------------------------------------------------------")


# # =======================
# # 使用方式
# # =======================
# if __name__ == "__main__":
#     input_dir = "你指定的資料夾路徑"
#     generate_text_from_folder(input_dir)


In [16]:
# =======================
# 使用方式
# =======================
input_dir = "data"
output_dir = "sentence"
generate_text_from_folder(input_dir, output_dir)

在資料夾 data 中找到 2 個 CSV 檔

讀取: vl_all_data_YM_convert.csv   → 偵測到 target_key = YM
→ 已輸出: sentence/vl_all_data_YM_sentence.csv
-----------------------------------------------------------
讀取: vl_all_data_YS_convert.csv   → 偵測到 target_key = YS
→ 已輸出: sentence/vl_all_data_YS_sentence.csv
-----------------------------------------------------------
